In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,60,2024-03-03,1,117.084746,97.771186,0.535932,9.994915,24.867797,0.236915,119.400000,...,10.585000,22.055000,0.197817,118.985000,0,3,6,2,23,2024
1,60,2024-03-03,3,119.101695,99.120339,0.556085,12.033898,27.547458,0.186898,118.954237,...,10.591525,24.423729,0.198559,123.554237,0,3,6,2,10,2024
2,59,2024-03-03,1,117.844828,97.167241,0.568155,11.213793,24.779310,0.216638,121.063793,...,12.763333,22.953333,0.220933,116.811667,0,3,6,2,13,2024
3,60,2024-03-03,-1,112.457627,99.613559,0.536746,13.045763,24.886441,0.195153,112.393220,...,12.528333,26.050000,0.225250,114.540000,0,3,6,2,9,2024
4,61,2024-03-03,-3,107.616667,97.141667,0.527350,11.813333,21.720000,0.168633,110.493333,...,11.321667,25.111667,0.185183,115.318333,0,3,6,2,5,2024
5,61,2024-03-03,-2,113.550000,95.545000,0.538117,11.071667,29.588333,0.203867,118.930000,...,11.627119,24.179661,0.187644,117.179661,0,3,6,3,20,2024
6,62,2024-03-03,-1,123.557377,101.672131,0.582115,11.214754,23.644262,0.176426,121.527869,...,12.570000,22.635000,0.172050,110.858333,0,3,6,2,12,2024
7,60,2024-03-03,-1,121.237288,99.137288,0.578712,10.511864,20.588136,0.205542,121.461017,...,12.591525,24.793220,0.241627,119.281356,0,3,6,3,21,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-03,PHI,DAL,2.140000,1.689655,0
1,2024-03-03,GSW,BOS,4.600000,1.212766,0
2,2024-03-03,LAC,MIN,2.140000,1.740741,0
3,2024-03-03,DET,ORL,5.400000,1.166667,0
4,2024-03-03,CHO,TOR,3.400000,1.335570,0
5,2024-03-03,NYK,CLE,3.050000,1.400000,0
6,2024-03-03,IND,SAS,1.400000,3.050000,1
7,2024-03-03,OKC,PHO,1.408163,3.000000,0


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-03,PHI,DAL,2.140000,1.689655,0
1,2024-03-03,GSW,BOS,4.600000,1.212766,0
2,2024-03-03,LAC,MIN,2.140000,1.740741,0
3,2024-03-03,DET,ORL,5.400000,1.166667,0
4,2024-03-03,CHO,TOR,3.400000,1.335570,0
5,2024-03-03,NYK,CLE,3.050000,1.400000,0
6,2024-03-03,IND,SAS,1.400000,3.050000,1
7,2024-03-03,OKC,PHO,1.408163,3.000000,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-03,PHI,DAL,2.140000,1.689655,0
1,2024-03-03,GSW,BOS,4.600000,1.212766,0
2,2024-03-03,LAC,MIN,2.140000,1.740741,0
3,2024-03-03,DET,ORL,5.400000,1.166667,0
4,2024-03-03,CHO,TOR,3.400000,1.335570,0
5,2024-03-03,NYK,CLE,3.050000,1.400000,0
6,2024-03-03,IND,SAS,1.400000,3.050000,1
7,2024-03-03,OKC,PHO,1.408163,3.000000,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-03,PHI,DAL,2.140000,1.689655,0
1,2024-03-03,GSW,BOS,4.600000,1.212766,0
2,2024-03-03,LAC,MIN,2.140000,1.740741,0
3,2024-03-03,DET,ORL,5.400000,1.166667,0
4,2024-03-03,CHO,TOR,3.400000,1.335570,0
5,2024-03-03,NYK,CLE,3.050000,1.400000,0
6,2024-03-03,IND,SAS,1.400000,3.050000,1
7,2024-03-03,OKC,PHO,1.408163,3.000000,0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-03,PHI,DAL,2.140000,1.689655,0.0
1,2024-03-03,GSW,BOS,4.600000,1.212766,0.0
2,2024-03-03,LAC,MIN,2.140000,1.740741,1.0
3,2024-03-03,DET,ORL,5.400000,1.166667,0.0
4,2024-03-03,CHO,TOR,3.400000,1.335570,0.0
5,2024-03-03,NYK,CLE,3.050000,1.400000,0.0
6,2024-03-03,IND,SAS,1.400000,3.050000,1.0
7,2024-03-03,OKC,PHO,1.408163,3.000000,1.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-03,PHI,DAL,2.140000,1.689655,0.0
1,2024-03-03,GSW,BOS,4.600000,1.212766,0.0
2,2024-03-03,LAC,MIN,2.140000,1.740741,1.0
3,2024-03-03,DET,ORL,5.400000,1.166667,0.0
4,2024-03-03,CHO,TOR,3.400000,1.335570,0.0
5,2024-03-03,NYK,CLE,3.050000,1.400000,0.0
6,2024-03-03,IND,SAS,1.400000,3.050000,1.0
7,2024-03-03,OKC,PHO,1.408163,3.000000,1.0
